In [36]:
import ollama
import os, json, time, textwrap, re
from pathlib import Path
import requests  
import wbpy
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.stattools import durbin_watson

# создание в рабочей директории папки для сохранения результатов
Path("outputs").mkdir(exist_ok=True)
# выбор модели
model_name = "llama3.2" 
print("Используемая модель:", model_name)

#____ОПРЕДЕЛЕНИЕ ПОКАЗАТЕЛЕЙ И СТРАН____
# коды показателей из World Bank API
indicator_y = {'gdp_per_capita_growth': 'NY.GDP.PCAP.KD.ZG'} # рост ВВП на душу населения (%)
indicators_x = {
    'foreign_direct_investment': 'BX.KLT.DINV.WD.GD.ZS', # прямые иностранные инвестиции, нетто-приток (% от ВВП)
    'export_growth': 'NE.EXP.GNFS.KD.ZG'                 # рост экспорта товаров и услуг (%)
}
indicator_dict = {**indicator_y, **indicators_x}

# коды стран из World Bank API
country_list = [
    'RU',  # Россия
    'US',  # США
    'DE',  # Германия
    'JP',  # Япония
    'BR',  # Бразилия
    'IN',  # Индия
    'FR',  # Франция
    'GB',  # Великобритания
    'CA',  # Канада
    'AU'   # Австралия
]

def get_wb_data(indicators, countries):
    """
    Функция для получения значений из World Bank API
    по заданным показателям и странам
    """
    # параметры запроса
    base_url = "http://api.worldbank.org/v2/country/"
    params = {
        'format': 'json',
        'per_page': 100,
        'date': '2010:2023'  # данные за период
    }
    data_list = []
    
    # данные для каждого показателя из списка
    for country in countries:
        for key, indicator_code in indicators.items():
            url = f"{base_url}{country}/indicator/{indicator_code}"
            response = requests.get(url, params=params)
            if response.status_code == 200: #200-статус успешного запроса
                data = response.json()[1]  # элемент [0] содержит метаданные, реальные данные в элементе [1]
                # преобразование в список значений (игнорирование пропусков)
                for item in data:
                    if item['value'] is not None:  # Игнорируем пропуски
                        row = {
                            # 'country': item['country']['value'],
                            'country_code': item['country']['id'],
                            'year': item['date'],
                            'indicator': key,
                            'value': item['value']
                        }
                        data_list.append(row)
            else:
                print(f"Ошибка при получении данных для {key} в стране {country}: {response.status_code}")
    # DataFrame
    df = pd.DataFrame(data_list)
    # трансформация в широкий формат 
    df_wide = df.pivot_table(
        index=['country_code', 'year'],
        columns='indicator',
        values='value'
    ).reset_index()
    # удаление строк с пропущенными значениями
    df_wide = df_wide.dropna()
    return df_wide


#____ВЫЗОВ ФУНКЦИИ, ПОЛУЧЕНИЕ ДАННЫХ____
# данные
df = get_wb_data(indicator_dict, country_list)
# результат
print("Размер датасета:", df.shape)
print("\nПервые 10 строк датасета:")
print(df.head(10))
# в CSV
df.to_csv('wb_dataset.csv', index=False)
print("\nДатасет сохранен в 'wb_dataset.csv'")


#____РЕГРЕССИОННЫЙ АНАЛИЗ (OLS-регрессия)____

# константа для свободного члена
X = sm.add_constant(df[list(indicators_x.keys())])
y = df[list(indicator_y.keys())]
model = sm.OLS(y, X)
results = model.fit()
print('\n\nОбщий результат OLS:')
print(results.summary())

def format_coef(coef):
    """Форматирует коэффициент по знаку"""
    if coef < 0:
        return f"- {abs(coef):.3f}"
    else:
        return f"+ {coef:.3f}"

# коэффициенты
b0 = results.params.get("const", 0)  # с защитой от отсутствия константы
# форматирование константы (при наличии начального плюса, убирается)
const_str = f"{b0:.3f}".lstrip('+')
# части уравнения для X переменных
equation_parts = []
# обработка каждой X переменной из словаря
for var_name in indicators_x.keys():
    if var_name in results.params:
        coef_value = results.params[var_name]
        coef_str = format_coef(coef_value)
        equation_parts.append(f"{coef_str}*{var_name}")
    else:
        print(f"Предупреждение: Переменная {var_name} не найдена в результатах регрессии")

# полное уравнение
y_var_name = list(indicator_y.keys())[0]  # название Y переменной
eqn = f"{y_var_name} = {const_str} " + " ".join(equation_parts)
print("\nУравнение регрессии:\n", eqn)

# p-values (значимость коэффициентов)
pvals = results.pvalues.to_dict()
print("\np-values по коэффициентам:", {k: round(v, 6) for k, v in pvals.items()})


#____КАЧЕСТВО РЕГРЕССИОННОЙ МОДЕЛИ (10 ключевых метрик)____

y_true = y.values                    # фактические значения
y_pred = results.fittedvalues.values # предсказанные моделью значения
resid  = results.resid.values        # остатки (разница между фактом и прогнозом)
n = len(y_true)
k = X.shape[1]                       # число параметров (включая const)

# базовые метрики
r2   = results.rsquared                 # объясненная дисперсия
r2a  = results.rsquared_adj             # R² с поправкой на количество предикторов
mae  = np.mean(np.abs(y_true - y_pred)) # средняя абсолютная ошибка
mse  = np.mean((y_true - y_pred)**2)    # средняя квадратичная ошибка
rmse = np.sqrt(mse)                     # корень из MSE

# MAPE — cредняя абсолютная процентная ошибка 
mask = (y_true != 0).flatten() # пропускает нулевые значения (деление на 0)
mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if mask.any() else np.nan
# RSE - остаточная стандартная ошибка (стандартное отклонение остатков)
sse = np.sum(resid**2)
rse = np.sqrt(sse / (n - k))

aic = results.aic           # информационный критерий Акаике 
bic = results.bic           # Байесовский информационный критерий
dw  = durbin_watson(resid)  # тест на автокорреляцию остатков (Durbin-Watson)

metrics = {
    "equation": eqn,
    "R2": r2,
    "Adj_R2": r2a,
    "MAE": mae,
    "MSE": mse,
    "RMSE": rmse,
    "MAPE_pct": mape,
    "RSE": rse,
    "AIC": aic,
    "BIC": bic,
    "Durbin_Watson": dw,
    "F_stat": results.fvalue,     # F-статистика всей модели
    "F_pvalue": results.f_pvalue  # p-значение F-теста
}
print('\n\nИспользуемые метрики качества регрессионной модели:')
print(json.dumps({k: (round(v, 6) if isinstance(v, (int, float, np.floating)) else v) for k, v in metrics.items()}, ensure_ascii=False, indent=2))


#____ОБРАБОТКА РЕЗУЛЬТАТОВ OLS С ПОМОЩЬЮ ollama____
# cводка для промпта: уравнение + ключевые числа
lines = []
lines.append(f"Уравнение: {eqn}")
lines.append(f"R2={r2:.3f}, Adj_R2={r2a:.3f}, RMSE={rmse:.3f}, MAE={mae:.3f}, MAPE={mape:.2f}%")
lines.append(f"AIC={aic:.3f}, BIC={bic:.3f}, RSE={rse:.3f}, DW={dw:.3f}")
lines.append(f"F={results.fvalue:.3f}, p(F)={results.f_pvalue:.3g}")
lines.append("p-values по коэффициентам: " + ", ".join([f"{k}={pvals[k]:.3g}" for k in pvals]))
summary_block = "\n".join(lines)

# промт
prompt_ols = (
"Ты — короткий русскоязычный аналитик. По регрессионной сводке ниже сформируй 3–6 пунктов:\n"
"- интерпретация знаков и значимости коэффициентов;\n"
"- адекватность модели (R2, Adj_R2, RMSE/MAE, MAPE);\n"
"- риски: автокорреляция (DW), переобучение (AIC/BIC), возможные упущенные факторы.\n"
"Формат: маркированный список, очень кратко.\n\n"
+ summary_block
)

# ответ модели
resp = ollama.generate(
    model=model_name,
    prompt=prompt_ols,
    options={"temperature": 0.3, "top_p": 0.9, "seed": 11, "num_predict": 350}
)
analysis_ols = resp["response"].strip()
print('\n\nОТВЕТ МОДЕЛИ:')
print(analysis_ols)

Используемая модель: llama3.2
Размер датасета: (139, 5)

Первые 10 строк датасета:
indicator country_code  year  export_growth  foreign_direct_investment  \
0                   AU  2010       4.710113                   3.094839   
1                   AU  2011       0.851116                   4.688815   
2                   AU  2012       4.603465                   3.720199   
3                   AU  2013       5.305440                   3.453928   
4                   AU  2014       6.074987                   4.304707   
5                   AU  2015       6.890783                   3.470209   
6                   AU  2016       6.412458                   3.560566   
7                   AU  2017       5.499422                   3.636090   
8                   AU  2018       4.126135                   4.250333   
9                   AU  2019       3.920744                   2.781968   

indicator  gdp_per_capita_growth  
0                       0.642043  
1                       0.993947